# First-Class Functions

Functions in Python are first-class objects. Programming language theorists define a “first-class object” as a program entity that can be:
- Created at runtime
- Assigned to a variable or element in a data structure
- Passed as an argument to a function
- Returned as the result of a function

## Treating a Function Like an Object

The console session in Example 5-1 shows that Python functions are objects. Here we create a function, call it, read its __doc__ attribute, and check that the function object itself is an instance of the function class

In [1]:
def factorial(n):
    """
    Returns n!
    """
    return 1 if n < 2 else n * factorial(n-1)

In [2]:
factorial(42)

1405006117752879898543142606244511569936384000000000

In [3]:
factorial.__doc__

'\n    Returns n!\n    '

In [4]:
type(factorial)

function

`We can assign it a variable fact and call it through that name.` We can also `pass factorial as an argument to map`. The map function returns an iterable where each item is the result of the application of the first argument (a function) to succesive elements of the second argument (an
iterable), range(10) in this example.

In [5]:
fact = factorial
fact

<function __main__.factorial(n)>

In [6]:
fact(5)

120

In [7]:
map(factorial, range(11))

In [8]:
list(map(fact, range(11)))

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

## Higher-Order Functions

`A function that takes a function as argument or returns a function as the result is a higher-order function.` One example is map, shown in Example 5-2. Another is the builtin function sorted: an optional key argument lets you provide a function to be applied to each item for sorting, as seen in “list.sort and the sorted Built-In Function” on page 42

### Modern Replacements for map, filter, and reduce

Functional languages commonly offer the map, filter, and reduce higher-order functions (sometimes with different names). The map and filter functions are still builtins in Python 3, but since the introduction of list comprehensions and generator expressions, they are not as important. `A listcomp or a genexp does the job of map and filter combined, but is more readable`

In [9]:
list(map(fact, range(6)))

[1, 1, 2, 6, 24, 120]

In [10]:
[fact(n) for n in range(6)]

[1, 1, 2, 6, 24, 120]

## Anonymous Functions

The lambda keyword creates an anonymous function within a python expression.

However, the simple syntax os ppython limits the body of lambda functions to be pure expression. In other words,, the body of a lambda cannot make assignments or us any other python statment such as while, try,  etc.

The best use of anonymous functions is in the context of an argument list.

In [11]:
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']
sorted(fruits, key=lambda word: word[::-1])

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

Outside the limited context of arguments to higher-order functions, anonymous functions are rarely useful in Python.

## The Seven Flavors of Callable Objects

The call operator `()` may be applied to other objects beyond user defined functions. To determine whether an object is callable, use the `callable()` built-in function.

The python data model documentation lists seven callable types:
1. User-Defined functions
    - created with `def` or lambda expressions
2. BUilt-in functions
    - A function implemented in C(for CPython) like len or time.strftime
3. Built-in method
    - Methods implemented in C like `dict.get`
4. Methods
    - Funtions defined in the body of a class
5. Classes
    - When invoked a class runs `__new__` method to create an instance, then `__init__` to initialize it, and finally the instance is returned to the caller. Because there is no new operator in python, calling a class is like calling a function
6. Class instances
    - If a classdefines a `__call__`, then its instances may be invoked as functions
7. Generator functions
    - Functions or method that use the `yield` keyword. Whenn called, generator functions return a generator object

In [12]:
[callable(obj) for obj in (abs, str, 13)]

[True, True, False]

## User-Defined Callable Types

Not only are python functions real objects, but arbitrary objects may also be made to behave like functions. Implementing a `__call__` method is all it takes.

In [14]:
import random

class BingoCage:
    
    def __init__(self, items) -> None:
        self._items = list(items)
        random.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
        
    def __call__(self):
        return self.pick()

1. `__init__` accepts any iterable; building a local copy prevents unexpected side effects on any list passed as an argument.
2. shuffle is guaranteed to work because self._items is a list.
3. The main method.
4. Raise exception with custom message if self._items is empty.
5. Shortcut to bingo.pick(): bingo()

In [15]:
bingo = BingoCage(range(3))

In [16]:
bingo()

0

In [17]:
bingo.pick()

1

A class implementing `__call__` is an eaasy ay  to create function-like objects that have some internal state that must be kep across invocations, like remaining items in the BingoCage. An example is a decorator. `Decorator` must be functions, but it is sometimes convenient to be able to "remember" something between calls of the decorator (eg., for memoization- caching the results of expensive computations for later use)


## Function Introspection 

Function objects have many attributes beyond __doc__. See what the dir function reveals about our factorial

In [21]:
print(dir(factorial))

['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


Most of these attributes are common to python objects in general. 

Like the instances of a plain user-defined class, a function uses the `__dict__` attribute to store user attributes assigned to it. This is useful as a primitive form of annotation. Assigning arbitrary attributes to functions is not a very common practice in general,

1. Create bare user-defined class.
2. Make an instance of it.
3. Create a bare function.
4. Using set difference, generate a sorted list of the attributesthat exist in a function but not in an instance of a bare class.

In [23]:
class C:
    pass

obj = C()

def func():
    pass

print(sorted(set(dir(func)) - set(dir(obj))))

['__annotations__', '__call__', '__closure__', '__code__', '__defaults__', '__get__', '__globals__', '__kwdefaults__', '__name__', '__qualname__']


## From Positional To Keyword-Only Parameters

One of the best features of Python functions is the extremely flexible parameter handling mechanism, enhanced with keyword-only arguments in Python 3. Closely related are the use of * and ** to “explode” iterables and mappings into separate arguments when we call a function.

In [30]:
def tag(name, *content, cls=None, **attrs):
    """Generate one or more html tags"""
    if cls:
        attrs['class'] = cls
    if attrs:
        attr_str = ''.join(' %s="%s"' % (attr, value)
                                        for attr, value
                                        in sorted(attrs.items()))
        
    else: 
        attr_str = ''
    if content:
        print(content)
        return '\n'.join('<%s%s>%s</%s>' %
                    (name, attr_str, c, name) for c in content)
    else:
        return '<%s%s />' % (name, attr_str)

In [26]:
tag('br')

'<br />'

In [31]:
print(tag('p', 'hello', 'world'))

('hello', 'world')
<p>hello</p>
<p>world</p>


In [29]:
tag('p', 'hello', id=33) 

'<p id="33">hello</p>'

In [32]:
my_tag = {'name': 'img', 'title': 'Sunset Boulevard',
 'src': 'sunset.jpg', 'cls': 'framed'}

tag(**my_tag)

'<img class="framed" src="sunset.jpg" title="Sunset Boulevard" />'

1. A single positional argument produces an empty tag with that name.
Any number of arguments after the first are captured by *content as a tuple.
2. Keyword arguments not explicitly named in the tag signature are captured by **attrs as a dict.
3. The cls parameter can only be passed as a keyword argument.
4. Even the first positional argument can be passed as a keyword when tag is called.
5. Prefixing the my_tag dict with ** passes all its items as separate arguments, which are then bound to the named parameters, with the remaining caught by **attrs.

Keyword-only arguments are a new feature in Python 3. In Example 5-10, the cls parameter can only be given as a keyword argument—it will never capture unnamed positional arguments. `To specify keyword-only arguments when defining a function, name them after the argument prefixed with *. `If you don’t want to support variable positional arguments but still want keyword-only arguments, put a * by itself in the signature, like this:

In [33]:
def f(a, *, b):
    return a, b

f(1,b=3)

(1, 3)

Note that keyword-only arguments do not need to have a default value: they can be mandatory, like b in the preceding example

## Retrieving Information About Parameters

Within a function object, the `__defaults__` attribute holds a tuple with the default values of positional and keyword arguments. The defaults for keyword-only arguments appear in `__kwdefaults__`. The names of the arguments, however, are found within the `__code__` attribute, which is a reference to a code object with many attributes of its own

In [34]:
def clip(text, max_len=80):
    """Return text clipped at the last space before or after max_len
    """
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else:   
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None: # no spaces were found
        end = len(text)
    return text[:end].rstrip()

In [35]:
clip.__defaults__

(80,)

In [36]:
clip.__code__ 

<code object clip at 0x000001F7710512F0, file "<ipython-input-34-d9fbaf10c083>", line 1>

In [37]:
clip.__code__.co_varnames

('text', 'max_len', 'end', 'space_before', 'space_after')

In [39]:
clip.__code__.co_argcount

2

As you can see, this is not the most convenient arrangement of information. The argument names appear in `__code__`.co_varnames, but that also includes the names of the local variables created in the body of the function. Therefore, the argument names are the first N strings, where N is given by `__code__`.co_argcount which—by the way—
does not include any variable arguments prefixed with * or **. The default values are identified only by their position in the `__defaults__` tuple, so to link each with the respective argument, you have to scan from last to first. In the example, we have two
arguments, text and max_len, and one default, 80, so it must belong to the last argument, max_len. This is awkward

Fortunately, there is a better way: the inspect module.


In [40]:
from inspect import signature

sig = signature(clip)
sig

<Signature (text, max_len=80)>

In [41]:
str(sig)

'(text, max_len=80)'

In [42]:
for name, param in sig.parameters.items():
    print(param.kind, ':', name, '=', param.default)

POSITIONAL_OR_KEYWORD : text = <class 'inspect._empty'>
POSITIONAL_OR_KEYWORD : max_len = 80


This is much better. inspect.signature returns an inspect.Signature object, which has a parameters attribute that lets you read an ordered mapping of names to inspect.Parameter objects. Each Parameter instance has attributes such as name, default, and kind. The special value inspect._empty denotes parameters with no default, which makes sense considering that None is a valid—and popular—default value.

The kind attribute holds one of five possible values from the _ParameterKind class:

1. POSITIONAL_OR_KEYWORD
    - A parameter that may be passed as a positional or as a keyword argument (mostPython function parameters are of this kind).
2. VAR_POSITIONAL
    - A tuple of positional parameters
3. VAR_KEYWORD
    - A dict of keyword parameters
4. KEYWORD_ONLY
    - A keyword-only parameter (new in Python 3)
5. POSITIONAL_ONLY
    - A positional-only parameter; currently unsupported by Python function declaration syntax, but exemplified by existing functions implemented in C—like divmod—that do not accept parameters passed by keyword

Besides name, default, and kind, inspect.Parameter objects have an annotation attribute that is usually inspect._empty but may contain function signature metadata provided via the new annotations syntax in Python 3 (annotations are covered in the next section)

An inspect.Signature object has a bind method that takes any number of arguments and binds them to the parameters in the signature, applying the usual rules for matching actual arguments to formal parameters. 


In [44]:
sig = signature(tag)

my_tag = {'name': 'img', 'title': 'Sunset Boulevard',
                'src': 'sunset.jpg', 'cls': 'framed'}

bound_args = sig.bind(**my_tag)
bound_args

<BoundArguments (name='img', cls='framed', attrs={'title': 'Sunset Boulevard', 'src': 'sunset.jpg'})>

In [45]:
for name, value in bound_args.arguments.items():
    print(name, '=', value)

name = img
cls = framed
attrs = {'title': 'Sunset Boulevard', 'src': 'sunset.jpg'}


In [46]:
del my_tag['name'] 
bound_args = sig.bind(**my_tag) 

TypeError: missing a required argument: 'name'

1. Get the signature from tag function
2. Pass a dict of arguments to .bind()
3. An inspect.BoundArguments object is produced.
4. Iterate over the items in bound_args.arguments, which is an          OrderedDict, to display the names and values of the arguments
5. Remove the mandatory argument name from my_tag.
6. Calling sig.bind(**my_tag) raises a TypeError complaining of the missing name parameter


This example shows how the Python data model, with the help of inspect, exposes the same machinery the interpreter uses to bind arguments to formal parameters in function calls.

Frameworks and tools like IDEs can use this information to validate code. Another feature of Python 3, function annotations, enhances the possible uses of this, as we will see next

## Function Annotations

Each argument in the function declaration may have an annotation expression preceded by :. If there is a default value, the annotation goes between the argument name and the = sign. To annotate the return value, add -> and another expression between the ) and the : at the tail of the function declaration. The expressions may be of any type.

No processing is done with the annotations. They are merely stored in the `__annotations__` attribute of the function, a dict:

In [48]:
def foo(text: str, num: float, length: 'int > 0' = 8) -> float:
    pass

foo.__annotations__

{'text': str, 'num': float, 'length': 'int > 0', 'return': float}

The only thing Python does with annotations is to store them in the `__annotations__` attribute of the function. Nothing else: no checks, enforcement, validation, or any other action is performed. In other words, annotations have no meaning to the Python interpreter. They are just metadata that may be used by tools, such as IDEs, frameworks, and decorators.

In [49]:
sig = signature(foo)
sig.return_annotation

float

In [50]:
for param in sig.parameters.values():
    note = repr(param.annotation).ljust(13)
    print(note, ':', param.name, '=', param.default)

<class 'str'> : text = <class 'inspect._empty'>
<class 'float'> : num = <class 'inspect._empty'>
'int > 0'     : length = 8


## Packages for Functional Programming


### The operator Module

Often in functional programming it is convenient to use an arithmetic operator as a function. For example, suppose you want to multiply a sequence of numbers to calculate factorials without using recursion. To perform summation, you can use sum, but there is no equivalent function for multiplication. You could use reduce

In [51]:
from functools import reduce

def fact(n):
    return reduce(lambda a, b: a*b, range(1, n+1))

In [52]:
fact(9)

362880

To save you the trouble of writing trivial anonymous functions like lambda: a, b: a*b, the operator module provides function equivalents for dozens of arithmetic operators.

In [53]:
from operator import mul

def fact(n):
    return reduce(mul, range(1, n+1))

In [54]:
fact(9)

362880

Another group of one-trick lambdas that operator replaces are functions to pick items from sequences or read attributes from objects: itemgetter and attrgetter actually build custom functions to do that.

In [55]:
metro_data = [
 ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
 ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
 ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
 ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
 ('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
 ]

In [56]:
from operator import itemgetter

for city in sorted(metro_data, key=itemgetter(1)):
    print(city)

('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833))
('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889))
('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
('Mexico City', 'MX', 20.142, (19.433333, -99.133333))
('New York-Newark', 'US', 20.104, (40.808611, -74.020386))


If you pass multiple index arguments to itemgetter, the function it builds will return tuples with the extracted values:

In [57]:
cc_name = itemgetter(1, 0)

for city in metro_data:
    print(cc_name(city))

('JP', 'Tokyo')
('IN', 'Delhi NCR')
('MX', 'Mexico City')
('US', 'New York-Newark')
('BR', 'Sao Paulo')


Because itemgetter uses the [] operator, it supports not only sequences but also mappings and any class that implements `__getitem__`

A sibling of `itemgetter` is `attrgetter`, which creates functions to extract object attributes by name. If you pass attrgetter several attribute names as arguments, it also returns a tuple of values. In addition, if any argument name contains a . (dot), attrgetter navigates through nested objects to retrieve the attribute

In [59]:
from collections import namedtuple

LatLong = namedtuple('LatLong', 'lat long')
Metropolis = namedtuple('Metropolis', 'name cc pop coord')


metro_areas = [Metropolis(name, cc, pop, LatLong(lat, long))
               for name, cc, pop, (lat, long) in metro_data]

metro_areas[0]

Metropolis(name='Tokyo', cc='JP', pop=36.933, coord=LatLong(lat=35.689722, long=139.691667))

In [60]:
metro_areas[0].coord.lat

35.689722

In [61]:
from operator import attrgetter

name_lat = attrgetter('name', 'coord.lat')

for city in sorted(metro_areas, key=attrgetter('coord.lat')):
    print(name_lat(city))

('Sao Paulo', -23.547778)
('Mexico City', 19.433333)
('Delhi NCR', 28.613889)
('Tokyo', 35.689722)
('New York-Newark', 40.808611)


1. Use namedtuple to define LatLong.
2. Also define Metropolis.
3. Build metro_areas list with Metropolis instances; note the nested tuple unpacking to extract (lat, long) and use them to build the LatLong for the coord attribute of Metropolis.
4. Reach into element metro_areas[0] to get its latitude.
5. Define an attrgetter to retrieve the name and the coord.lat nested attribute.
6. Use attrgetter again to sort list of cities by latitude.
7. Use the attrgetter defined in to show only city name and latitude.

In [64]:
import operator
print([name for name in dir(operator) if not name.startswith('_')])

['abs', 'add', 'and_', 'attrgetter', 'concat', 'contains', 'countOf', 'delitem', 'eq', 'floordiv', 'ge', 'getitem', 'gt', 'iadd', 'iand', 'iconcat', 'ifloordiv', 'ilshift', 'imatmul', 'imod', 'imul', 'index', 'indexOf', 'inv', 'invert', 'ior', 'ipow', 'irshift', 'is_', 'is_not', 'isub', 'itemgetter', 'itruediv', 'ixor', 'le', 'length_hint', 'lshift', 'lt', 'matmul', 'methodcaller', 'mod', 'mul', 'ne', 'neg', 'not_', 'or_', 'pos', 'pow', 'rshift', 'setitem', 'sub', 'truediv', 'truth', 'xor']


Most of the 52 names listed are self-evident. T`he group of names prefixed with i and the name of another operator—e.g., iadd, iand, etc.—correspond to the augmented assignment operators—e.g., +=, &=, etc`. These change their first argument in place, if it is mutable; if not, the function works like the one without the i prefix: it simply returns
the result of the operation


Of the remaining operator functions, `methodcaller` is the last we will cover. It is somewhat similar to attrgetter and itemgetter in that it creates a function on the fly. The function it creates calls a method by name on the object given as argument.

In [65]:
from operator import methodcaller

s = "The time has come"

upcase = methodcaller('upper')

upcase(s)

'THE TIME HAS COME'

In [66]:
hiphenate = methodcaller('replace', ' ', '-')
hiphenate(s)

'The-time-has-come'

### Freezing Arguments with functools.partial

The functools module brings together a handful of higher-order functions. The best known of them is probably reduce.Of the remaining functions in functools, the most useful is `partial and its variation, partialmethod.`

functools.partial is a higher-order function that allows partial application of a function. Given a function, a partial application produces a new callable with some of the arguments of the original function fixed. This is useful to adapt a function that takes
one or more arguments to an API that requires a callback with fewer arguments.

In [67]:
from operator import mul
from functools import partial

triple = partial(mul, 3)

triple(7)

21

In [68]:
list(map(triple, range(1, 10))) 

[3, 6, 9, 12, 15, 18, 21, 24, 27]

1. Create new triple function from mul, binding first positional argument to 3.
2. Test it.
3. Use triple with map; mul would not work with map in this example.

partial takes a callable as first argument, followed by an arbitrary number of positional and keyword arguments to bind.

# Chatper Summary

The goal of this chapter was to explore the first-class nature of functions in Python. `The main ideas are that you can assign functions to variables, pass them to other functions, store them in data structures, and access function attributes, allowing frameworks and tools to act on that information`. Higher-order functions, a staple of functional programming, are common in Python—even if the use of map, filter, and reduce is not as frequent as it was—thanks to list comprehensions (and similar constructs like generator expressions) and the appearance of `reducing built-ins like sum, all, and any.` The `sorted, min, max built-ins, and functools.partial are examples of commonly used higher-order functions in the language.
`
Callables come in seven different flavors in Python, from the simple functions created with lambda to instances of classes implementing `__call__`. They can all be detected by the `callable() built-in.` Every callable supports the same rich syntax for declaring formal parameters, including `keyword-only parameters and annotations`—both new features introduced with Python 3.

Python functions and their annotations have a rich set of attributes that can be read with the help of the `inspect module, which includes the Signature.bind method to apply the flexible rules` that Python uses to bind actual arguments to declared parameters


Lastly, we covered some functions from the `operator module and functools.partial`, which facilitate functional programming by minimizing the need for the functionally challenged lambda syntax